In [1]:
import numpy as np  
import pandas as pd  
import tensorflow as tf  
import os  
import cv2 
from glob import glob  
from pathlib import Path  

In [2]:
!pip install ultralytics

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 17.6 MB/s eta 0:00:00a 0:00:01


In [3]:
import os

dataset_path = '/kaggle/input/potato-leaf-disease-dataset'

for root, dirs, files in os.walk(dataset_path):
    print(f"Root: {root}")
    print(f"Directories: {dirs}")
    print("-" * 50)

Root: /kaggle/input/potato-leaf-disease-dataset
Directories: ['Potato___healthy', 'Potato___Late_blight', 'Potato___Early_blight']
--------------------------------------------------
Root: /kaggle/input/potato-leaf-disease-dataset/Potato___healthy
Directories: []
--------------------------------------------------
Root: /kaggle/input/potato-leaf-disease-dataset/Potato___Late_blight
Directories: []
--------------------------------------------------
Root: /kaggle/input/potato-leaf-disease-dataset/Potato___Early_blight
Directories: []
--------------------------------------------------


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Dataset and output paths
dataset_path = '/kaggle/input/potato-leaf-disease-dataset'
output_path = '/kaggle/working/potato_leaf_yolo_main'

# Create YOLO folders
os.makedirs(f'{output_path}/train/images', exist_ok=True)
os.makedirs(f'{output_path}/train/labels', exist_ok=True)
os.makedirs(f'{output_path}/val/images', exist_ok=True)
os.makedirs(f'{output_path}/val/labels', exist_ok=True)

# Class mapping
class_names = ['Potato___healthy', 'Potato___Late_blight', 'Potato___Early_blight']
class_map = {name: i for i, name in enumerate(class_names)}

# Process each class folder
for class_name in class_names:
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_folder):
        print(f"Warning: Folder '{class_folder}' does not exist.")
        continue

    # Get all image files
    images = [f for f in os.listdir(class_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    # Split data into train and validation (80-20 split)
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    # Move files to respective YOLO folders
    for img_set, set_type in [(train_imgs, 'train'), (val_imgs, 'val')]:
        for img in img_set:
            # Copy image
            src = os.path.join(class_folder, img)
            dst = os.path.join(output_path, set_type, 'images', img)
            shutil.copy(src, dst)

            # Generate YOLO-format label file
            label_path = os.path.join(output_path, set_type, 'labels', img.rsplit('.', 1)[0] + '.txt')
            with open(label_path, 'w') as f:
                class_id = class_map[class_name]
                # Placeholder for bounding box (replace these with actual normalized coordinates)
                # Format: <class_id> <x_center> <y_center> <width> <height>
                f.write(f'{class_id} 0.5 0.5 0.5 0.5\n')

# Define the name of the data.yaml file
data_yaml_name = 'data.yaml'
data_yaml_path = os.path.join(output_path, data_yaml_name)

# Create a data.yaml file for YOLO training
with open(data_yaml_path, 'w') as f:
    f.write(f"""train: {output_path}/train/images
val: {output_path}/val/images

nc: {len(class_names)}
names: {class_names}
""")

print(f"Dataset is ready for YOLO training. '{data_yaml_name}' created at {data_yaml_path}.")

Dataset is ready for YOLO training. 'data.yaml' created at /kaggle/working/potato_leaf_yolo_main/data.yaml.


In [6]:
import os
import cv2
from collections import Counter

# Function to check image sizes in a folder
def check_image_sizes(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    size_counter = Counter()

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        if image is not None:
            height, width, _ = image.shape
            size_counter[(width, height)] += 1
        else:
            print(f"Could not read image: {image_file}")

    return size_counter

# Paths to train and val folders
train_images_path = os.path.join(output_path, 'train/images')
val_images_path = os.path.join(output_path, 'val/images')

# Check image sizes
train_sizes = check_image_sizes(train_images_path)
val_sizes = check_image_sizes(val_images_path)

# Print the results
print("Train Image Sizes:")
for size, count in train_sizes.items():
    print(f"Size {size}: {count} images")

print("\nValidation Image Sizes:")
for size, count in val_sizes.items():
    print(f"Size {size}: {count} images")


Train Image Sizes:
Size (256, 256): 1721 images

Validation Image Sizes:
Size (256, 256): 431 images


In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n') 
results = model.train(data='/kaggle/working/potato_leaf_yolo_main/data.yaml', epochs=50, imgsz=128,batch=16,device=0)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 75.1MB/s]


Ultralytics 8.3.73 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/potato_leaf_yolo_main/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

...

                   all        431        431      0.837      0.563      0.599      0.262



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50      0.22G      1.475        1.4      1.251         16        128: 100%|██████████| 108/108 [00:09<00:00, 11.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00,  8.84it/s]


                   all        431        431      0.748      0.805      0.807      0.626

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50      0.22G      1.421      1.285      1.231         35        128:  48%|████▊     | 52/108 [00:04<00:04, 11.52it/s]

In [ ]:
from IPython.display import Image as IPyImage
import os
HOME = os.getcwd()
from IPython.display import Image as IPyImage
IPyImage(filename=f'/kaggle/working/runs/detect/train/results.png', width=1000)

pot